In [7]:
import numpy as np
import pandas
import itertools
import inspect
import uuid
import ray

uuid.uuid3(uuid.NAMESPACE_DNS, 'python.org')

UUID('6fa459ea-ee8a-3ca4-894e-db77e160355e')

In [27]:

class Flow:
    def __init__(self, model, mesh, name, shape):
        self.shape = shape # 1 Dimension for now
        self.model = model
        self.mesh = mesh
        self.computed = np.zeros(shape, dtype = bool)
        self.values = np.zeros(shape, dtype = float) * np.nan # not very clean
        self.compute_mode = False
        
    def __getitem__(self, *args):
        if self.compute_mode & (not self.computed[args[0]]):
            self.formula(args[0])
        return self.values.__getitem__(args[0])
    
    def __setitem__(self, *args):
        self.values[args[0]] = args[1]
        self.computed[args[0]] = True
    
    def slide_window(self):
        pass
    
    def compute(self, index):
        """
        Args::
        - index : index of the value to compute. Compute all values if index == -2.
                  Compute the last value if index == -1

        if index == -1:
        return self[self.shape - 1]
        elif index == -2:
        ranges = (range(i) for i in self.shape)
        for index in itertools.product(*ranges):
          self[index]
        return self[self.shape - 1]
        """
        self.compute_mode = True
        return self[index]
  
    def formula(self, index):
        s = self
        m = self.mesh
        top = self.model
        
        # Formula stuff...
        self[index] = None # some stuff...
    def resize(self, shape):
        self.computed = np.zeros(shape, dtype = bool)
        self.values = np.zeros(shape, dtype = float) * np.nan

class Mesh:
    def __init__(self, model, name, shape, shape_names):
        self.shape = shape
        self.model = model
        self.shape_names = shape_names
        
        self.flow = {}
        self.mesh = {}
        
    def compute(self, index):
        pass
        
    def __getitem__(self, item):
        return self.flow[item]
    
    def resize(self, shape):
        for key in self.flow.keys():
            self.flow[key].resize(shape)


class Report:
    def __init__(self, model, name):
        self.model = model
        self.name = name
    def export(self):
        pass
    def compute(self):
        pass



class Model:
    mesh = {}
    report = {}
    def __init__(self):
        pass
    def __getitem__(self, item):
        return self.mesh[item]

In [4]:

        
class FlowTest(Flow):
    def __init__(self, model, mesh, name, shape):
        super().__init__(model, mesh, name, shape)
        self.T1, self.T2 = self.shape[0], self.shape[1]
        
    def formula(self, index):
        t1, t2, mp = index
        T1, T2 = self.T1, self.T2
        if (t1 == 0) & (t2 == 0):
            self[t1, t2, mp] = 0
        elif (t2 == 0):
            self[t1, t2, mp] = self[t1-1, T2-1, mp] + 1
        else:
            self[t1, t2, mp] = self[t1, t2-1, mp] + 1
    

flow = FlowTest(None, None, "FlowTest", (10, 10, 3))
print(flow.compute_mode)
print(flow.compute_mode)
flow.compute((5, 3, 1))
flow.values[:,: , 2]

False
False


array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]])

In [5]:


class Report_central(Report):
    def __init__(self, model, name):
        super().__init__(self, model, name)
    def compute(self):
        model["prod_tra"].compute(-1)

In [9]:
class Credit(Mesh):
    def __init__(self, model, name, shape, shape_names):
        super().__init__(model, name, shape, shape_names)
        
        self.flow["flow_test"] = FlowTest(model, self, "flow_test", shape)
        #self.mesh["submesh1"] = Mesh(model, "submesh1", [10, 2, 1], ["time", "etape", "credit", "submesh1"])

        
class Prod_tra(Mesh):
    def __init__(self, *args):
        super().__init__(*args)
        
class Goc_insu(Mesh):
    def __init__(self, *args):
        super().__init__(*args)
        
class Goc_reass(Mesh):
    def __init__(self, *args):
        super().__init__(*args)


class Pv_prod_tra(Mesh):
    def __init__(self, *args):
        super().__init__(*args)
        
    
class PVIF(Model):
    T = 10
    substeps = 2
    def __init__(self):
        super().__init__()
        self.mesh["credit"] = Credit(self, "credit", [self.T, self.substeps, 1], ["time", "etape", "mp"])
        self.mesh["prod_tra"] = Prod_tra(self, "prod_tra", [self.T, self.substeps, 1], ["time", "etape", "mp"])
        self.mesh["goc_insu"] = Goc_insu(self, "goc_insu", [self.T, self.substeps, 1], ["time", "etape", "mp"])
        self.mesh["goc_reass"] = Goc_reass(self, "goc_reass", [self.T, self.substeps, 1], ["time", "etape", "mp"])
        self.mesh["pv_prod_tra"] = Pv_prod_tra(self, "pv_prod_tra", [self.T, self.substeps, 1], ["time", "etape", "mp"])
        self.report["central"] = Report(self, "central")
        self.data = {}

    def on_data_fed(self):
        """
        Resize data and then additionnal operations
        """
        self["credit"].resize((self.T, self.substeps, len(self.data["mp"]) ))

    def feed_data_central(self):
        """
        Just assign values to self.data
        """
        self.data["mp"] = pandas.DataFrame({"id": list(range(10)), 
                                          "csr": [100] * 10, 
                                          "prod_activated": [True] * 3 + [False] * 7})
        self.on_data_fed()

    def fetch_report_central(self):
        self.report["central"].compute()
        self.report["central"].export()
    

pvif = PVIF()
pvif.feed_data_central()

print(pvif["credit"]["flow_test"][2, 1, 0])
pvif["credit"]["flow_test"][2, 1, 0] = 10
print(pvif["credit"]["flow_test"][2, 1, 0])

print(pvif["credit"].model["credit"])
print(pvif["credit"])

nan
10.0


In [13]:
pvif["credit"]["flow_test"].compute((2, 1, 2))

5.0

In [19]:
pvif["credit"]["flow_test"].values[:,:,2]

array([[ 0.,  1.],
       [ 2.,  3.],
       [ 4.,  5.],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan]])

In [ ]:
pvif["credit"]["flow_test"].values[:,:,2]

In [21]:
class test:
    def __getitem__(self, index):
        print(index)

In [25]:
a = test()
a[5:8, 8]

(slice(5, 8, None), 8)


In [24]:
slice(5, 8, None)

slice(5, 8, None)

In [26]:
slice(5, 8, None)

slice(5, 8, None)